In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_moons
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=16hnqHPHGeIsv71pzu2iWu1l6UsD4tBaf' # The shareable link

In [ ]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

16hnqHPHGeIsv71pzu2iWu1l6UsD4tBaf


In [ ]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Cleaned_data.csv')  
df = pd.read_csv('Cleaned_data.csv')
# Dataset is now stored in a Pandas Dataframe
df=pd.DataFrame(df)

In [ ]:
df1 = df[df.TARGET == 1].sample(10000)
df2 = df[df.TARGET == 0].sample(40000)



In [ ]:
(df1.TARGET == 1).sum()
(df2.TARGET == 0).sum()


frames = [df1, df2]

df = pd.concat(frames)


(df.TARGET == 1).sum()
(df.TARGET == 0).sum()

df = pd.get_dummies(df)

df = df.drop(columns=["CODE_GENDER_F", "FLAG_OWN_CAR_Y", "FLAG_OWN_CAR_Y"],axis=1)


In [ ]:
link = 'https://drive.google.com/open?id=1rG7Teq_NwZ0wP063KMilyWuxexgpUujy' # The shareable link

In [ ]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='


1rG7Teq_NwZ0wP063KMilyWuxexgpUujy


In [ ]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Sampled_data.csv') 

In [ ]:
df.to_csv('Sampled_data.csv')

In [ ]:
df=pd.read_csv('Sampled_data.csv')

In [ ]:

X_train, X_test, y_train, y_test = \
        train_test_split(df.loc[:, df.columns != 'TARGET'], 
                         df['TARGET'], test_size=.33, random_state=42)

In [ ]:
clf = RandomForestClassifier(max_depth=4, n_estimators=4, 
                             max_features='sqrt', random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred) 
print('f1 score Random Forest is = ' + str(f1))


f1 score Random Forest is = 0.0035492457852706297


In [ ]:
cm_random_forest_classifier = confusion_matrix(y_test,y_pred)
print(cm_random_forest_classifier)


[[13125     1]
 [ 3368     6]]


In [ ]:
numerator = cm_random_forest_classifier[0][0] + cm_random_forest_classifier[1][1]
denominator = sum(cm_random_forest_classifier[0]) + sum(cm_random_forest_classifier[1])
acc_rfc = (numerator/denominator) * 100
print("Accuracy : ",round(acc_rfc,2),"%")


Accuracy :  79.58 %


In [ ]:
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.8264675056950568, 0.5008510600056125, 0.444902344962928, None)

In [ ]:
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train,y_train)
y_pred_xgb = xgb_classifier.predict(X_test)

In [ ]:
cm_xgb_classifier = confusion_matrix(y_test,y_pred_xgb)
print(cm_xgb_classifier)

[[12852   274]
 [ 2923   451]]


In [ ]:
numerator = cm_xgb_classifier[0][0] + cm_xgb_classifier[1][1]
denominator = sum(cm_xgb_classifier[0]) + sum(cm_xgb_classifier[1])
acc_xgb = (numerator/denominator) * 100
print("Accuracy : ",round(acc_xgb,2),"%")


Accuracy :  80.62 %


In [ ]:
f1 = f1_score(y_test, y_pred_xgb) 
print('f1 score XGB is = ' + str(f1))

f1 score XGB is = 0.22005367162722614


In [ ]:
#precision_score(y_test, y_pred_xgb, average='macro')
#print(precision_score)
precision_recall_fscore_support(y_test, y_pred_xgb, average='macro')


(0.7183878900486365, 0.5563973176492563, 0.5547173309521896, None)

In [ ]:
import lightgbm as lgb

In [ ]:
d_train = lgb.Dataset(X_train, label=y_train)

In [ ]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 100
params['min_data'] = 5000
params['max_depth'] = 1000
clf = lgb.train(params, d_train, 10000)

In [ ]:
#Prediction
y_pred=clf.predict(X_test)
#convert into binary values
for i in range(0,999):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

In [ ]:
cm = confusion_matrix(y_test, y_pred.round())
print("Confusion Matrix LGB:")
print(cm)


#Accuracy

accuracy=accuracy_score(y_pred.round(),y_test)*100
print("Accuracy LGB: ",accuracy,"%")


Confusion Matrix LGB:
[[12816   310]
 [ 2913   461]]
Accuracy LGB:  80.46666666666667 %


Accuracy LGB:  80.46666666666667 %


In [ ]:
f1 = f1_score(y_test, y_pred.round()) 
print('f1 score of lgb is = ' + str(f1))

f1 score of lgb is = 0.14089709762532981


In [ ]:
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.8264675056950568, 0.5008510600056125, 0.444902344962928, None)

In [ ]:
 from sklearn.svm import SVC

In [ ]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)